In [1]:
# TODO - plot function ASTs

# TODO: FIRST THING TOMORROW - need to draw the GNN diagram, use this as a BLUEPRINT
# for the model design

# - I expect it to evolve as I go, but need to START SOMEWHERE
# - Annotate what the feature vectors look like for nodes
# - How many hops away do we need to gather information (GNN depth)?
# - What virtual edges might we insert? (e.g. connect all instances of target variable)
# - Should we MASK OUT the target variable?

# also...
# - look at function proto recovery accuracy

In [2]:
import os
import pandas as pd
from pathlib import Path

# Execute this notebook headlessly using something like:
#
# EXP_FOLDER=~/test_builds/coreutils.exp/ jupyter nbconvert --to html --execute characterize_dataset.ipynb --no-input
#

MANUAL_EXP_FOLDER = Path.home()/'test_builds'/'astera3.exp'

# take the env-var-specified experiment if present
EXP_FOLDER = Path(os.environ['EXP_FOLDER']) if 'EXP_FOLDER' in os.environ else MANUAL_EXP_FOLDER

In [3]:
# import wildebeest as wdb
# wdb.get_experiment_names()
ast_json = EXP_FOLDER/'rundata'/'run1'/'0.fighter'/'ast_dumps'/'stripped'/'Func143603-FUN_00143603.json'
[x for x in (EXP_FOLDER/'rundata'/'run1'/'0.fighter'/'ast_dumps'/'stripped').iterdir() if '143603' in x.name]


[PosixPath('/home/cls0027/test_builds/astera3.exp/rundata/run1/0.fighter/ast_dumps/stripped/Func143603-FUN_00143603.json')]

# Prepare data for PyG Model
*Sticking this here for now for sake of time...*

We need to **quickly** write the code to take our data tables and convert each variable into the format ready to go for PyG

In [4]:
node_kind_ids = {
    'ArraySubscriptExpr': 0,
    'BinaryOperator': 1,
    'BreakStmt': 2,
    'BuiltinType': 3,
    'CallExpr': 4,
    'CaseStmt': 5,
    'CharacterLiteral': 6,
    'CompoundStmt': 7,
    'ConstantArrayType': 8,
    'ConstantExpr': 9,
    'CStyleCastExpr': 10,
    'DeclRefExpr': 11,
    'DeclStmt': 12,
    'DoStmt': 13,
    'EnumDecl': 14,
    'EnumConstantDecl': 15,
    'EnumType': 16,
    'FieldDecl': 17,
    'FloatingLiteral': 18,
    'ForStmt': 19,
    'FunctionDecl': 20,
    'FunctionType': 21,
    'GotoStmt': 22,
    'IfStmt': 23,
    'IntegerLiteral': 24,
    'LabelStmt': 25,
    'MemberExpr': 26,
    'NullNode': 27,
    'ParenExpr': 28,
    'ParmVarDecl': 29,
    'PointerType': 30,
    'RecordDecl': 31,
    'ReturnStmt': 32,
    'StringLiteral': 33,
    'StructField': 34,
    'StructType': 35,
    'SwitchStmt': 36,
    'TranslationUnitDecl': 37,
    'Type': 38,
    'TypedefDecl': 39,
    'TypedefType': 40,
    'UnaryOperator': 41,
    'ValueDecl': 42,
    'VarDecl': 43,
    'VoidType': 44,
    'WhileStmt': 45,
}

In [5]:
import astlib
from astlib.color_ast import highlight_var_refs, highlight_khop_neighborhood
from astlib.find_all_references import *

from typing import Dict, Tuple


run_folder = EXP_FOLDER/'rundata'/'run1'

# list((EXP_FOLDER/'rundata'/'run1').iterdir())
# list(run_folder.glob('*.csv'))

# def generate_dtpredict_dataset(run_folder:Path):
locals_df = pd.read_csv(run_folder/'locals.csv')

# NOTE: need to go back to AST JSON directly to read it back in (we have to use the whole thing anyway)
# NOTE: we could be a bit more efficient by passing the ASTs we already have in-memory to this function
# instead of reading them back again from the file...
# TODO: would be nice to just have the AST filenames here in the functions table though
funcs_df = pd.read_csv(run_folder/'functions.csv')


# addr = locals_df.FunctionStart.unique()[0]
# ast_file = funcs_df[funcs_df.FunctionStart==addr].AstJson_Strip.iloc[0]

# ast, slib = astlib.json_to_ast(ast_file)

# # locals_df[locals_df.FunctionStart]
# for i, x in enumerate(locals_df.groupby(['BinaryId', 'FunctionStart'])):
#     # print(x)
#     if i > 5:
#         break

# x[1]

# AHA! DataFrameGroupBy object behaves like an iterable of tuples:
    # [0]: tuple of values for the columns you grouped by
    # [1]: corresponding data frame with this subset of the data

##########################################################
# TODO: write a function that converts the local variables into pyg format for a given function dataframe
# - then I can pass this function to a groupby() that splits the data by unique binary/function combos
#   and it will work as intended
##########################################################
df_list = locals_df.groupby(['BinaryId', 'FunctionStart']).pipe(lambda gb: [x[1] for x in gb])
# --------------------
# TODO: for all locals in this function (start here, we'll see how it evolves)
# 1. Find all references to the local (see below)
# 2. Extract those statements in which the reference "participates" (see below)
# 3. Join the target nodes all into one, preserving existing connections
#       - ok to do this "directly" on the data, we read it from the file and won't modify original file
# 4. Write this resulting graph to a file ready to go for pyg
#       - how does pyg want this? just a json file? look at pyg tutorials
#
#       >>> only need to save the data/features we need for the model (and just enough metadata
#           to refer back to the func/var of interest)
##########################################################

# TODO: convert this to a function we call using pipe() above
df = df_list[33]

addr = df.FunctionStart.iloc[0]
ast_file = funcs_df[funcs_df.FunctionStart==addr].iloc[0].AstJson_Strip

ast, slib = astlib.json_to_ast(ast_file)
df.drop(columns=['LocType_Debug','LocRegName_Debug','LocOffset_Debug','LocType_Strip','LocRegName_Strip','LocOffset_Strip'])

# -------------------------------------------
# TODO: for each local... i in range(len(df))
name_strip = df.iloc[0].Name_Strip
# -------------------------------------------

fbody = ast.inner[-1].inner[-1]

# NOTE: this is the simplest approach - just taking k-hop neighborhoods for
# each reference
# --> however, limiting neighborhoods to staying within the statement seems
#     better since other statements are independent!
#     (and we know this...why confuse the model?)
# decl_ref_nodes = FindAllVarRefs(name_strip).visit(fbody)
# node = decl_ref_nodes[0]
# for node in decl_ref_nodes:
#     display(node.parent.parent.parent.render(format_node=highlight_khop_neighborhood(node, 2, 'red')))

# statements = FindAllStatementsContainingVar(name_strip).collect_statement_set(fbody)
# statements

# zero out parent connection to ensure we don't reach outside
# the statement
# for s in statements:
#     s.statement_node.parent = None

# for s in statements:
    # display(s.statement_node.render(format_node=highlight_khop_neighborhood(s.refexprs, 2)))

# ----------------------------------------------------------------
# TODO: go back to pyg tutorials and see what format pyg wants...then make this fit!
# ----------------------------------------------------------------
# 1. node list
#   - target node at 0th position
#   - assign indices 0 -> n-1 (node.pyg_idx = next_idx++) as we add nodes
# 2. edge_index
#   - refer to node endpoints by index (above)
#   - add bidirectional edges probably...

# FIXME: later on, we will add these things, but for now:
# - No edge types
# - NODE TYPE is only node feature

# s = statements[2]
# TODO: need to map possible truth data types to enums (we will then one-hot them)


######## OLD: don't need to FIND statements ahead of time now
# # TODO: we don't even need statements anymore! lol
# # --> just find all variable references and don't COLLECT any neighbors
# #     outside the statement (dynamically we can check with node.is_statement)
# statements = FindAllStatementsContainingVar(self.var_name).collect_statement_set(fbody)
# # zero out parent connection to ensure we don't reach outside the statement
# for s in statements:
#     s.statement_node.parent = None
# ref_exprs = [r for s in statements for r in s.refexprs]

# each refexpr is an independent sample
# ref_exprs = [r for s in statements for r in s.refexprs]

####################### ---------------------######################
####################### TODO: pick up here...######################
####################### ---------------------######################
# each ref_expr will be joined into node #0 in our graph

import torch
from torch.nn import functional as F

MAX_HOPS = 3

def encode(node:ASTNode) -> torch.Tensor:
    '''Encodes an ASTNode into a feature vector'''
    return F.one_hot(torch.tensor(node_kind_ids[node.kind]), len(node_kind_ids))

class VariableGraphBuilder:
    '''
    Builds the variable graph that will be used as input to the model.

    Currently supports only locals and params
    '''
    def __init__(self, var_name:str, ast:ASTNode, slib:Dict[int, astlib.StructDef]):
        '''
        var_name: Name of the target variable
        ast: AST for the function this variable resides in
        slib: Struct library for this AST (as returned by astlib.json_to_ast() with
              the AST object)
        '''
        self.__reset_state()
        self.var_name = var_name

        self.ast = ast
        self.slib = slib

    def __reset_state(self):
        self._next_idx = 0
        self.node_list = []
        self.edge_index = []

    def build_variable_graph(self, max_hops:int) -> Tuple[torch.Tensor, torch.Tensor]:
        '''
        Generates the variable graph and returns it as a
        [node_list, edge_index] tuple of tensors
        '''
        self.__reset_state()

        fbody = self.ast.inner[-1].inner[-1]
        ref_exprs = FindAllVarRefs(self.var_name).visit(fbody)

        # each refexpr is an independent sample that needs to be merged
        # into our target node 0

        # 1. add node 0/merge ref_exprs into special target node
        for r in ref_exprs:
            r.pyg_idx = self._next_idx
        self._next_idx += 1

        self.node_list = [encode(ref_exprs[0])]  # pick one and encode it, they are identical

        # edge index starts out as a list of strings of the form "<start_idx>,<stop_idx>"
        # so we can prevent adding duplicate edges. Then we convert to tensor form once finished
        self.edge_index = []

        # add other nodes by following edges up to MAX HOPS
        for r in ref_exprs:
            # collect subgraph connected to r (we've already got the reference node captured)
            self.collect_node_neighbors(r, max_hops)

        # collect edge indices into flat list, then reshape into (N, 2)
        flat_list = [int(idx) for edge_str in self.edge_index for idx in edge_str.split(',')]
        N = int(len(flat_list)/2)
        self.edge_index = torch.tensor(flat_list, dtype=torch.long).reshape((N, 2))

        # torch-ify :)
        self.node_list = torch.stack(self.node_list)
        self.edge_index = self.edge_index.t().contiguous()

        return self.node_list, self.edge_index

    def add_node(self, node:ASTNode):
        if not hasattr(node, 'pyg_idx'):
            # this is a new node - add it
            node.pyg_idx = self._next_idx
            self._next_idx += 1
            self.node_list.append(encode(node))

    def _get_edge_string(self, start_idx:int, stop_idx:int):
        return f'{start_idx},{stop_idx}'

    def add_edge(self, start_node:ASTNode, end_node:ASTNode, bidirectional:bool):
        fwd_edge = self._get_edge_string(start_node.pyg_idx, end_node.pyg_idx)
        if fwd_edge not in self.edge_index:
            self.edge_index.append(fwd_edge)

        if bidirectional:
            back_edge = self._get_edge_string(end_node.pyg_idx, start_node.pyg_idx)
            if back_edge not in self.edge_index:
                self.edge_index.append(back_edge)

    def collect_node_neighbors(self, node:ASTNode, k:int):
        '''
        Collect the k-hop neighborhood of node (not including node) staying
        within the current statement
        '''
        if k <= 0:
            return

        # if we are at a statement node, don't go up outside this statement
        if node.parent and not node.is_statement:
            self.add_node(node.parent)
            self.add_edge(node, node.parent, bidirectional=True)
            self.collect_node_neighbors(node.parent, k-1)

        for child in node.inner:
            self.add_node(child)
            self.add_edge(node, child, bidirectional=True)
            self.collect_node_neighbors(child, k-1)

# ref_exprs
# node_list = torch.stack([encode(x) for x in ref_exprs])
# node_list

In [6]:
# ast.inner[-1].inner[-1].inner[-3].render()
print(ast.inner[-1].inner[-1].inner[-3].inner[2].inner[1].parent)

In [7]:
builder = VariableGraphBuilder(name_strip, ast, slib)
node_list, edge_index = builder.build_variable_graph(MAX_HOPS)
node_list

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [12]:
edge_index

tensor([[ 0,  1,  1,  2,  0,  3,  3,  4,  4,  5,  4,  6,  3,  7,  7,  8,  0,  9,
          9, 10, 10,  0, 10, 11,  0, 12, 12, 13, 13, 14, 12, 15,  0, 16, 16, 17,
         17, 18, 17, 19,  0, 20, 20, 21, 21, 22, 21, 23,  0, 24, 24, 25, 25, 26,
         25, 20, 24, 27,  0, 28, 28, 29, 29, 30, 28, 31,  0, 32, 32, 33,  0, 34,
         34, 35, 35, 36, 34, 37,  0, 38, 38, 39, 38, 40, 38, 41, 38, 42, 38, 43,
         38, 44,  0, 45, 45, 46, 46, 47, 45, 48,  0, 49, 49, 50, 50, 51, 49, 52,
          0, 53, 53, 54, 54, 55, 53, 56],
        [ 1,  0,  2,  1,  3,  0,  4,  3,  5,  4,  6,  4,  7,  3,  8,  7,  9,  0,
         10,  9,  0, 10, 11, 10, 12,  0, 13, 12, 14, 13, 15, 12, 16,  0, 17, 16,
         18, 17, 19, 17, 20,  0, 21, 20, 22, 21, 23, 21, 24,  0, 25, 24, 26, 25,
         20, 25, 27, 24, 28,  0, 29, 28, 30, 29, 31, 28, 32,  0, 33, 32, 34,  0,
         35, 34, 36, 35, 37, 34, 38,  0, 39, 38, 40, 38, 41, 38, 42, 38, 43, 38,
         44, 38, 45,  0, 46, 45, 47, 46, 48, 45, 49,  0, 50, 49, 51

In [ ]:
import astlib
from astlib.color_ast import highlight_var_refs

ast, slib = astlib.json_to_ast(ast_json)
ast.inner[-1].inner[-1].inner[8].inner[0].inner

# if node.kind == highlight_kind:
#             attrs.font_color = highlight_color
# def highlight_var(varname:str):
#     def do_highlight(node, attrs):
#         if node.kind == 'DeclRefExpr' and node.referencedDecl.name == varname:
#             attrs.font_color = 'red'
#     return do_highlight

ast.render(format_node=highlight_var_refs('param_1'))

In [ ]:
pvdecl = ast.inner[-1].inner[0]
pvdecl.name

fbody = ast.inner[-1].inner[-1]
from astlib.find_all_references import *

res = FindAllVarRefs('param_1').visit(fbody)
# res[0].parent.parent.parent.parent.parent.parent
statements = FindAllStatementsContainingVar('param_1').collect_statement_set(fbody)
statements

In [ ]:
for s in statements:
    display(s.render(format_node=highlight_var_refs('param_1')))